In [2]:
import tiktoken             # 导入分词工具
import torch

In [3]:
# 自定义数据集

class DataLoaderLite():
    
    def __init__(self, B, T):
        self.B = B
        self.T = T

        # 从训练数据中加载数据
        with open('input.txt', 'r') as f:
            text = f.read()
        
        # 先进行分词
        enc = tiktoken.get_encoding('gpt2')
        tokens = enc.encode(text)

        # 将分词后的token转为tensor格式
        self.tokens = torch.tensor(tokens)
        
        # 打印显示有多少tokens
        print(f"loaded {len(self.tokens)} tokens")
        print(f"1 epoch = {len(self.tokens) // (B * T)} batchs")

        # 标记已经用过的数据
        self.current_position = 0

    def next_batch(self):
        B, T = self.B, self.T
        # 准备输入和标签
        buf = self.tokens[self.current_position : self.current_position+B*T+1]
        x = (buf[:-1]).view(B, T)
        y = (buf[1:]).view(B, T)

        # 下一批用新的数据
        self.current_position += B * T

        # 如果1个epoch用完了，则计数器归零
        if self.current_position + (B * T + 1) > len(self.tokens):
            self.current_position = 0
        
        return x, y


In [24]:
train_loader = DataLoaderLite(40, 320)
for i in range(1):
    x, y = train_loader.next_batch()
    z = train_loader.current_position
    print(z) 

loaded 338025 tokens
1 epoch = 26 batchs
12800
